In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns

import warnings
import yaml
import pygsp
import glob
import matplotlib

from tslearn.metrics import dtw
from tslearn.clustering import TimeSeriesKMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, Birch, BisectingKMeans, MiniBatchKMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

from plotly.subplots import make_subplots

from pyprojroot import here
from tsmoothie import LowessSmoother, ExponentialSmoother

pd.set_option('display.max_columns', None)

ROOT_DIR = str(here())
insar_path = ROOT_DIR + "/data/raw/insar/" 

pio.templates.default = 'plotly'
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
# PRE PROCCESSING

def interpolate_displacement(df):
    interpolated_df = df.set_index('timestamp').resample('6D').ffill()
    interpolated_df['displacement'] = (
                                       df[['timestamp','displacement']].set_index('timestamp')
                                                                       .resample('6D')
                                                                       .interpolate(method='linear')
                                      )
    return interpolated_df


def smoothing(frac):
    def smoothing_(x):
        lowess_smoother = LowessSmoother(smooth_fraction=frac, iterations=1) #0.075 
        lowess_smoother.smooth(x)
        return lowess_smoother.smooth_data[0]
    return smoothing_

In [ ]:
# Loading original file
# Put insar csv file here
filename = 'insar_file.csv'

df_orig = pd.read_csv(insar_path + filename)
#df_orig = pd.read_csv(insar_dir + "D1/L2B_037_0695_IW2_VV.csv") # Sensors Paper


# Visualization of file density
fig = px.density_heatmap(x=df_orig.longitude, y=df_orig.latitude, nbinsx = 100, nbinsy=100, width=600, height=600)
fig.show()

In [ ]:
# SELECT AND FORMAT DATA
# Here you can set the region you want to extract from the datafile by setting latitude and longitude boundaries

df = df_orig.copy()
lat_min, lat_max, lon_min, lon_max = (63.4182, 63.4220, 10.3858, 10.3946) # Sensors Paper

df = df[ (df.longitude>lon_min) & (df.longitude<=lon_max) &
            (df.latitude>lat_min) & (df.latitude<=lat_max)  ]

fig = px.density_heatmap(x=df.longitude, y=df.latitude, nbinsx = 100, nbinsy=100, width=500, height=00)
fig.show()

# Selection relevant columns
date_cols = sorted([col for col in df.columns if "20" in col]) #columns named after timestamps
keep_cols = date_cols #list with variables to keep from dataframe
id_cols = ['pid', 'latitude', 'longitude', 'easting', 'northing', 'mean_velocity']
keep_cols.extend(id_cols)
df = df[keep_cols]

# Formatting from wide to tall dataframe
# Uses a single column for timestamp and a column for displacement
# Number of rows = number of pixels * number of timestamps
df = df.melt(id_vars=id_cols, value_vars=date_cols,
                var_name='timestamp', value_name='displacement').sort_values('pid')
df.timestamp = pd.to_datetime(df.timestamp)


# Selecing time period: based on gap before 2016.06
df = df[df.timestamp>='2016-06-01'].copy()
df.reset_index(drop=True, inplace=True)
df.sort_values(['pid','timestamp'], inplace=True)

# CLUSTERING PIXELS (to work with smaller groups at once later)
# Simply gives each pixel a new attribute identifying it into a local cluster.
average_size = 1000 # Average size of clusters
nodes_full = df.drop_duplicates(['pid'])[['pid', 'easting','northing']]
nodes_full['cluster'] = KMeans(n_clusters=nodes_full.shape[0]//average_size).fit_predict(nodes_full[['northing','easting']])
df = df.merge(nodes_full[['pid','cluster']], how='left', on='pid')

print(f'{df.pid.nunique()} nodes')

In [ ]:
# INTERPOLATE MISSING TIMESTAMPS
df = (df.groupby('pid', as_index=False)
                .apply(interpolate_displacement)
                .reset_index().drop('level_0', axis=1)
                )

In [ ]:
# APPLY SMOOTHNESS
df['smoothed'] = df.groupby('pid',as_index=False).displacement.transform(smoothing(50/df.timestamp.nunique()))
# df['smooth60'] = df.groupby('pid',as_index=False).displacement.transform(smoothing(60/df.timestamp.nunique()))

In [ ]:
# SAVE

output_name = 'output_name.parq'
df.to_parquet(ROOT_DIR+f"/data/interim/{output_name}")